In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# **Lab 5:** GenAI for BigQuery
This lab will showcase some Generative AI features that can be utilized comfortably within BigQuery. While going through this lab you will learn:
* **Creating a model**: To be able to use GenAI we will create a model using BigQueryML syntax which will be remotely connected to our VertexAI platform.
* **Generation**: After successful model creation we will engineer a suitable prompt based on our data and use this to generate new text utilizing fitting BigQuery ML syntax as well.

What we'd like to achieve contentwise is, that we generate a message for each user, depending on churn or not, and let them know about their current gaming status.

### **Step 1:** Parameters and Authentication
Authenticate yourself against Google Cloud Platform.

In [1]:
project_id   = "<project-id>"
team_name    = "<team-name>"
location     = "us" #This is currently necessary
region       = "us-central1"

dataset_name = "datathon_ds_{}".format(team_name)
bucket_name  = "gs://{}_{}".format(project_id,dataset_name)

In [2]:
from google.cloud import bigquery
import google.auth

credentials, project_id = google.auth.default()
client = bigquery.Client(credentials=credentials, project=project_id)


### **Step 2:** Create full dataset
As a first step to incorporate GenAI features into BigQuery, we need the necessary data to work on.
For this we'll create another view of our raw data which incorporates the full dataset, as in Lab 1.

In [ ]:
# Create view for the full dataset
view_id  = "{}.{}.cc_full_dataset_for_genai".format(project_id, dataset_name)
view     = bigquery.Table(view_id)

view.view_query = f'''

  SELECT
    dem.*,
    IFNULL(beh.cnt_user_engagement, 0) AS cnt_user_engagement,
    IFNULL(beh.cnt_level_start_quickplay, 0) AS cnt_level_start_quickplay,
    IFNULL(beh.cnt_level_end_quickplay, 0) AS cnt_level_end_quickplay,
    IFNULL(beh.cnt_level_complete_quickplay, 0) AS cnt_level_complete_quickplay,
    IFNULL(beh.cnt_level_reset_quickplay, 0) AS cnt_level_reset_quickplay,
    IFNULL(beh.cnt_post_score, 0) AS cnt_post_score,
    IFNULL(beh.cnt_spend_virtual_currency, 0) AS cnt_spend_virtual_currency,
    IFNULL(beh.cnt_ad_reward, 0) AS cnt_ad_reward,
    IFNULL(beh.cnt_challenge_a_friend, 0) AS cnt_challenge_a_friend,
    IFNULL(beh.cnt_completed_5_levels, 0) AS cnt_completed_5_levels,
    IFNULL(beh.cnt_use_extra_steps, 0) AS cnt_use_extra_steps,
    ret.user_first_engagement,
    ret.churned
  FROM
    {dataset_name}.user_returninginfo ret
  LEFT OUTER JOIN
    {dataset_name}.user_demographics dem
  ON
    ret.user_pseudo_id = dem.user_pseudo_id
  LEFT OUTER JOIN
    {dataset_name}.user_aggregate_behavior beh
  ON
    ret.user_pseudo_id = beh.user_pseudo_id
  WHERE ret.bounced = 0
'''

# Create the view
view = client.create_table(view, exists_ok=True)
print(f"Created {view.table_type}: {str(view.reference)}")

Since further table alterations can not be done on views, we are going to create a new BigQuery Table according to the previously created view.

Please replace the placeholder for project ID and dataset name in full text within the following code.

In [ ]:
# Replace placeholders with your actual project ID and Dataset ID with your actual view name and desired new table name
view_id = '<project-id>.<dataset-name>.cc_full_dataset_for_genai'
table_id = '<project-id>.<dataset-name>.cc_full_dataset_table'

# Extract the SQL query from the view
query_job = client.query(f"""
    SELECT *
    FROM `{view_id}`
""")

# Create the table from the view query
table = bigquery.Table(table_id)
job_config = bigquery.QueryJobConfig(destination=table_id)

job = client.query(query_job.query, job_config=job_config)

# Wait for the job to complete
job.result()

print(f"Table {table_id} created successfully from view!")

### **Step 3**: The prompt

Now, to utilize GenAI features, we need a prompt.

For this there are two ways to go.

One possibility is that one can specify an additional "prompt" column in our dataset and insert the prompts there. The other method would not create an additional column and have another prompt variable to specifiy in the query.
Depending on your use case you could play around with both, but here we will use the method with an additional prompt column.

In [ ]:
#create a new column called 'prompt'
# replace project ID and dataset ID with your specific values in full text
%%bigquery --project $project_id
ALTER TABLE `<project-id>.<dataset-name>.cc_full_dataset_table`
ADD COLUMN prompt string;

After we successfully created the new prompt column we now specify how the prompt ist going to look like.

By default the same prompt would be inserted into each cell. If we are interested in creating a specific promt base on other column values we'd have to go into more advanced prompting, like in the following example.

In [ ]:
# replace project ID and dataset ID with your specific values in full text
%%bigquery --project $project_id

UPDATE `<project-id>.<dataset-name>.cc_full_dataset_table`
SET prompt = CASE
    WHEN churned = 1 THEN CONCAT("Please generate an uplifting message to tell the that they haven't quit the game and it seems that they are enjoying it. Please also include a reference to the number of user engagements which will follow after this prompt. (3 sentences max)", CAST (cnt_user_engagement AS STRING))
    ELSE CONCAT("Please generate a sad message to tell the user that it seems like they have quit the game. Then try encouraging them to rejoin the game again while including a reference to the number of user engagements which will follow after this prompt. (3 sentences max)", CAST (cnt_user_engagement AS STRING))
END
WHERE TRUE;

### **(Optional)**
You could try and incorporate the respective user's language in the prompt as well in a further try. Just concatenate a further column's value in the prompt and refer to it as well.

The user's country of origin is listed in the column 'country'.

### **Step 4** Create a model

After having our prompt column in place we need to create a model within BigQuery to be able to utilize GenAI capabilities.

Firstly, we need to create an external connection within BigQuery to connect remotely to Vertex AI models.

In [ ]:
#change project ID and connection name (choose a name for your connection)
!bq mk --connection --location='us' --project_id='<project-id>' \
    --connection_type=CLOUD_RESOURCE <connection-name>

We will now utilize this new connection to create a model.

The model **Text Bison** was chosen for this notebook, but feel free to play with other LLMs as well via their endpoints.

In [ ]:
# replace project ID and connection ID with your specific values in full text
%%bigquery --project $project_id

CREATE MODEL `<project-id>.<dataset-name>.<model-name>` #set your model's name
REMOTE WITH CONNECTION `<project-id>.us.<connection-name>`
OPTIONS (

  ENDPOINT = 'https://us-central1-aiplatform.googleapis.com/v1/projects/datajourney030524/locations/us-central1/publishers/google/models/text-bison:predict'
);

### **!!Important!!**
Before hopping into generation, we have to grant the generated service account from the new connection the necessary IAM permissions.

Go to the BigQuery page in the console and click on the just created connection. The page will show you a service account associated with it.

Insert this service account as the principal on the IAM page when clicking on 'Grant access'.

Grant this new principal access to the following:

* Vertex AI User
* BigQuery Connection User
* BigQuery Data Editor

### **Step 5** Generate message

Now it's time to start generating messages for every user based on their specific gaming behavior!

Since the original table has several ten thousand entries it will take quite a while to finish generating.

Therefore for demonstration purposes we'll create a quick table which only includes the first 100 entries from our original table.

In [ ]:
%%bigquery --project $project_id

CREATE TABLE `<project-id>.<dataset-name>.cc_full_dataset_table_100` AS
SELECT *
FROM `<project-id>.<dataset-name>.cc_full_dataset_table`
LIMIT 100;


And now the fun part! Let's start generating the messages. To see them in full, it's also useful to navigate to your BigQuery in the Cloud Console and query the following there as well.

In [ ]:
%%bigquery --project $project_id

SELECT * FROM

ML.GENERATE_TEXT(
  MODEL `<project-id>.<dataset-name>.model1_test`,
  TABLE `<project-id>.<dataset-name>.cc_full_dataset_table_100`,
  STRUCT (
    0.2 AS temperature,
    75 AS max_output_tokens,
    0.3 AS top_p,
    TRUE AS flatten_json_output
  )
);